In [327]:
import pexpect
from pexpect import EOF, TIMEOUT
import sys

class DeviceBase(object):

    def __init__(self, params):

        self.params = params
        self.hostname = params.get("hostname","")
        self.port = params.get("port", 22)
        self.username = params.get("username", "lab")
        self.password = params.get("password", "lab")
        self.connect_type = params.get("connect_type", "ssh")
        self.device_type = params.get("device_type", "iosxr")
        self.debug = params.get("debug", 0)
        self.verbose = params.get("verbose", False)
        self.expect_list = []

    def _telnet(self):

        cmd = "telnet {hostname} {port}".format(hostname=self.hostname, port=self.port)
        return cmd

    def _ssh(self):
        ssh = "ssh {hostname} -l {username} " \
              " -p {port} " \
              "-o StrictHostKeyChecking=no " \
              "-o UserKnownHostsFile=/dev/null".format(username=self.username,
                                                       hostname=self.hostname,
                                                       port=self.port)
        return ssh

    def connect(self, timeout=5):
        if self.connect_type.lower() == "ssh":
            self.c =pexpect.spawn(self._ssh(), encoding="utf-8")

        elif self.connect_type.lower() == "telnet":
            self.c = pexpect.spawn(self._telnet(), encoding="utf-8")
        else:
            raise ("connect error")

        self.c.delaybeforesend = 0.05
        self.c.timeout = timeout
        self.c.logfile = sys.stdout

        return self.c

    def get_prompt(self):
        pass

    def send_command(self, cmd, prompt="", timeout=10):
        if  prompt:            
            self.prompt = prompt
        self.c.sendline(cmd)
        print("sent command %s" %cmd)
        result = []
        try:
            i = self.c.expect(self.prompt, timeout=timeout)
            if i == 0:                
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
        except TIMEOUT:
                print("connect to %s timeout" % self.hostname)
        except Exception as e:
            print(e)
            
        return result
            
    def login(self, prompt=r"[>|#|$]\s?$"):
        self.expect_list = []
        self.expect_list.append(r"(?i)username[:]?\s*$")    # username
        self.expect_list.append(r"(?i)login[:]?\s*$")       # login
        self.expect_list.append(r"(?i)password[:]?\s*$")    # password
        # self.expect_list.append(r"secret:\s*$")           # IOS-XR init config
        self.expect_list.append(prompt)
        self.c.send("\n\r")
        for _ in range(0, 3):
            result = []

            try:
                i = self.c.expect(self.expect_list, timeout=5)
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
                if i == 0 or i == 1:
                    self.c.sendline(self.username)
                elif i == 2:
                    self.c.sendline(self.password)
                elif i == 3:
                    break
            except EOF:
                break
            except TIMEOUT:
                print("connect to %s timeout" % self.hostname)
                break

        if result[0] < 3:
            print(result)
            print("username or password error")
            return result

        self.get_prompt()

        self._set_terminal_length_zero()
        return result

    def _set_terminal_length_zero(self):
        pass

    def logout(self):
        if self.c:
            self.c.terminate()

    def __del__(self):
        self.logout()


In [328]:
class IOSXR(DeviceBase):
    
    def get_prompt(self):

        self.c.sendline()
        self.c.expect(r"#$")
        prompt = str(self.c.before) + str(self.c.after)
        prompt = prompt.splitlines()
        prompt = prompt[-1]
        self.prompt = prompt
        return self.prompt

    def to_config_mode(self):
        r = self.send_command("config terminal",prompt="config.*#$")
        print("into config mode")
        return r

    def send_config(self, cmd,prompt="config.*#$",commit=False):
        
        prompt = self.get_prompt()
        if "config" not in prompt:
            self.to_config_mode()   
        
        r = self.send_command(cmd,prompt="config.*#$")
        print("sent config line %s" %cmd)
        
        if commit:
            self.send_command("commit")
    
    def send_configs(self, cmds, prompt="config.*#$", commit=False):
        if isinstance(cmds,list):
            for cmd in cmds:
                self.send_config(cmd,prompt="config.*#$")
        elif isinstance(cmds, str):
            for cmd in cmds.splitlines():
                self.send_config(cmd,prompt)
        if commit:
            self.send_command("commit")
            
        
 
    def to_enable(self, prompt="\#$"):
        self.send_command("clear",prompt)
        self.send_command("end",prompt)
        

    def _set_terminal_length_zero(self):
        self.c.sendline("terminal length 0")
        try:
            i = self.c.expect(self.prompt)
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")

    def get_config(self):
        self.expect_list = []
        self.expect_list.append(self.prompt)
        print(self.prompt)
        result = []
        self.c.sendline("show running-config")
        try:
            i = self.c.expect(self.expect_list, timeout=5)
            if i == 0:
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")
        return result


In [360]:
if __name__ == "__main__":

    device = {"hostname": "172.20.0.1",
              "port":33281,
              "username": "admin",
              "password": "admin",
              "connect_type": "telnet",
              "device_type": "iosxr"}
    d = IOSXR(device)
    d.connect()
    d.login()
    
    



Trying 172.20.0.1...
Connected to 172.20.0.1.
Escape character is '^]'.
]0;xrv1
RP/0/0/CPU0:ios#
RP/0/0/CPU0:ios#

RP/0/0/CPU0:ios#terminal length 0
terminal length 0
Wed Jul 18 15:12:18.149 UTC
RP/0/0/CPU0:ios#

In [358]:
d.get_prompt()



RP/0/0/CPU0:ios(config-if)#

'RP/0/0/CPU0:ios(config-if)#'

In [361]:
d.send_config("interface gi0/0/0/0")



RP/0/0/CPU0:ios#config terminal
sent command config terminal
config terminal
Wed Jul 18 15:12:29.798 UTC
RP/0/0/CPU0:ios(config)#into config mode
interface gi0/0/0/0
sent command interface gi0/0/0/0
interface gi0/0/0/0
RP/0/0/CPU0:ios(config-if)#sent config line interface gi0/0/0/0


In [363]:
d.send_config("ipv4 address 11.1.1.1/30")



RP/0/0/CPU0:ios(config-if)#ipv4 address 11.1.1.1/30
sent command ipv4 address 11.1.1.1/30
ipv4 address 11.1.1.1/30
RP/0/0/CPU0:ios(config-if)#sent config line ipv4 address 11.1.1.1/30
commit
sent command commit
commit
Wed Jul 18 15:12:46.667 UTC
RP/0/0/CPU0:ios(config-if)#

In [350]:
d.prompt

'\\#$'

In [351]:
d.send_command("show run",timeout=5)

show run
sent command show run
show run
Wed Jul 18 15:06:13.354 UTC
Building configuration...
!! IOS XR Configuration 6.5.1.31I
!! Last configuration change at Wed Jul 18 04:21:27 2018 by admin
!
interface MgmtEth0/0/CPU0/0
 ipv4 address dhcp
!
interface GigabitEthernet0/0/0/0
 shutdown
!
interface GigabitEthernet0/0/0/1
 shutdown
!
interface GigabitEthernet0/0/0/2
 shutdown
!
interface GigabitEthernet0/0/0/3
 shutdown
!
interface GigabitEthernet0/0/0/4
 shutdown
!
interface GigabitEthernet0/0/0/5
 shutdown
!
interface GigabitEthernet0/0/0/6
 shutdown
!
interface GigabitEthernet0/0/0/7
 shutdown
!
interface GigabitEthernet0/0/0/8
 shutdown
!
interface GigabitEthernet0/0/0/9
 shutdown
!
interface GigabitEthernet0/0/0/10
 shutdown
!
ssh server vrf default
end

RP/0/0/CPU0:ios#

[0,
 'show run\r\n\rWed Jul 18 15:06:13.354 UTC\r\nBuilding configuration...\r\n!! IOS XR Configuration 6.5.1.31I\r\n!! Last configuration change at Wed Jul 18 04:21:27 2018 by admin\r\n!\r\ninterface MgmtEth0/0/CPU0/0\r\n ipv4 address dhcp\r\n!\r\ninterface GigabitEthernet0/0/0/0\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/1\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/2\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/3\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/4\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/5\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/6\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/7\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/8\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/9\r\n shutdown\r\n!\r\ninterface GigabitEthernet0/0/0/10\r\n shutdown\r\n!\r\nssh server vrf default\r\nend\r\n\r\nRP/0/0/CPU0:ios',
 '#']

In [349]:
d.to_enable()

clear
sent command clear
clear
RP/0/0/CPU0:ios(config)#end
sent command end
end
RP/0/0/CPU0:ios#